<a href="https://colab.research.google.com/github/MadsYDL/intro-to-software-development/blob/main/Boring%C3%987.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#OF UNITS
#density in pcf, mass in lbs


In [28]:
#slurry
rho_light = 110#lbs/ft
rho_tail = 140#lbs/ft
Q = 13 #bbl/min


#geometry
Annulus_ID_OD = [[13.375,17.5],[13.375,19.124]]#inches
tail_D = 12.415#inches

#cement section specs
Ls = 3.2*np.array([25,25,25,1850-1150-25-25,250])#ft
section_rho_slurry = [rho_tail,rho_tail,rho_tail,rho_light,rho_light]#tail disk, tail annulus, plug, shielding annulus, connection annulus
config = ["disk","annulus","disk","annulus","annulus"]
df_sec = pd.DataFrame({"length":Ls,"rho":section_rho_slurry,"config":config})
df_sec['D'] = [17.5,-1,12.415,-1,-1]
df_sec['OD'] = [-1,17.5,-1,17.5,19.124]
df_sec['ID'] = [-1,13.375,-1,13.375,13.375]
df_sec['SF'] = [0,0.25,0,0.25,0]

#solid specs
m_s,rho_s = np.array([94]),np.array([198])#lbs,pcf


#liquid specs
rho_w = 62.4#pcf



df_sec

,length,rho,config,D,OD,ID,SF
0,80.0,140,disk,17.500,-1.000,-1.000,0.00
1,80.0,140,annulus,-1.000,17.500,13.375,0.25
2,80.0,140,disk,12.415,-1.000,-1.000,0.00
3,2080.0,110,annulus,-1.000,17.500,13.375,0.25
4,800.0,110,annulus,-1.000,19.124,13.375,0.00


In [29]:
#Vols
def annular_volume(L,OD,ID,SF=0):
  #return ft^3
  return L*(1+SF)*(np.pi/4)*(OD**2-ID**2)*0.0833333**2

def circular_volume(L,D,SF=0):
  return L*(1+SF)*np.pi/4*D**2*0.0833333**2

#slurry props

def V_w(m_solids,rho_solids,rho_slurry,rho_water,checker = True):
  if checker == True and rho_water in rho_solids:
    print("ERROR: only additatives and cement should be included in rho_solids and m_solids")
  return (sum(m_solids)-rho_slurry*sum(m_solids/rho_solids))/(rho_slurry-rho_water)


def slurry_volume_per_sack(m_solids, rho_solids, rho_slurry, rho_water):
    #input parameters on per sac basis
    Vwater = V_w(m_solids, rho_solids, rho_slurry, rho_water)
    solids_vol = np.sum(np.asarray(m_solids) / np.asarray(rho_solids))
    return solids_vol + Vwater


def cement_bags_needed_given_V(m_solids, rho_solids, rho_slurry, rho_water,V):
  slurry_vol_persack = slurry_volume_per_sack(m_solids, rho_solids, rho_slurry, rho_water)
  return np.ceil(V/slurry_vol_persack)




In [30]:
df_sec

,length,rho,config,D,OD,ID,SF
0,80.0,140,disk,17.500,-1.000,-1.000,0.00
1,80.0,140,annulus,-1.000,17.500,13.375,0.25
2,80.0,140,disk,12.415,-1.000,-1.000,0.00
3,2080.0,110,annulus,-1.000,17.500,13.375,0.25
4,800.0,110,annulus,-1.000,19.124,13.375,0.00


In [38]:
def compute_volume(row):
    if row['config'] == 'annulus':
        return annular_volume(row['length'], row['OD'], row['ID'], row['SF'])
    else:
        return circular_volume(row['length'], row['D'], row['SF'])


df_sec['vol per sack'] = slurry_volume_per_sack(m_s,rho_s,df_sec['rho'],rho_w)

df_sec['sacks_needed'] = df_sec['vol']/df_sec['vol per sack']
df_sec

,length,rho,config,D,OD,ID,SF,vol,vol per sack,sacks_needed
0,80.0,140,disk,17.500,-1.000,-1.000,0.00,133.626664,0.829585,161.076615
1,80.0,140,annulus,-1.000,17.500,13.375,0.25,69.463708,0.829585,83.733131
2,80.0,140,disk,12.415,-1.000,-1.000,0.00,67.252816,0.829585,81.068072
3,2080.0,110,annulus,-1.000,17.500,13.375,0.25,1806.056402,1.352432,1335.413951
4,800.0,110,annulus,-1.000,19.124,13.375,0.00,815.228465,1.352432,602.787080


In [47]:
Vtot = df_sec['vol'].sum()
print("amound of sacks of cement needed = ",Vtot,'ft3' )

amound of sacks of cement needed =  2891.628055271932 ft3


In [48]:
Q = 13*5.614 #ft3/min

thickening_time = Vtot/Q+60 #min
print("thickening time = ", thickening_time, "min")

thickening time =  99.62111281236376 min


In [49]:
sacks = df_sec['sacks_needed'].sum()
print("amound of sacks of cement needed = ",sacks )

amound of sacks of cement needed =  2264.0788483841234
